In [1]:
# import lib
from IPython.display import Latex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchsummary
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Function
import  visdom
import scipy.sparse as sp
import gurobipy as gp
import itertools
import pdb
import datetime
from random import choice
from gurobipy import GRB
import pickle

# Prepare data

In [2]:
def load_data(data_dir='2_layer_weights/32/'):
    dict_param = {}
    dict_param['w0'] = np.sign(np.load(data_dir+"w0.npy")).T
    _, dict_param['size_input'] = np.shape(dict_param['w0'])
    # we assume all hidden layers have the same number of neurons
    dict_param['w1'] = np.sign(np.load(data_dir+"w1.npy")).T  
    dict_param['size_hidden'] = np.shape(dict_param['w1'].T)
    dict_param['w2'] = np.sign(np.load(data_dir+"w2.npy")).T
    dict_param['size_output'],_ = np.shape(dict_param['w2'])

    # MNIST dataset
    test_dataset = torchvision.datasets.MNIST(root='data',
                                              train=False,
                                              transform=transforms.ToTensor())
    return dict_param, test_dataset

In [3]:
def set_param(dict_param, test_dataset, restriction=False,  flip_rate=0.5):
    x, label = choice(test_dataset)
    x = x.reshape(28*28).numpy()*2-1
    x = np.sign(x)
    x[x==0] = 1 
    w0,w1,w2 = (dict_param[name] for name in ['w0','w1','w2'])
    confidence_vec = w2@np.sign(dict_param['w1']@np.sign(dict_param['w0']@x))
    label_predicted = np.argmax(confidence_vec)
    if restriction is True:
        flips = 2*np.random.binomial(n=1, p=1-flip_rate, size=dict_param['size_input'])-1
        x_turb = x * flips
        target = np.argmax(w2@np.sign(dict_param['w1']@np.sign(dict_param['w0']@x_turb)))
        dict_param['set_const'] = np.nonzero(flips==1)[0]
        dict_param['set_var'] = np.nonzero(flips != 1)[0]
        print('the number of flips is {}'.format(len(dict_param['set_var'])))
    else:
        dict_param['set_var'] = range(dict_param['size_input'])
        dict_param['set_const'] = range(0)
        select_range = np.nonzero(confidence_vec<np.max(confidence_vec))[0] # avoid equal confidence level
        #target = (label_predicted + np.random.randint(9) + 1) % 10
        if len(select_range) == 0:
            raise Exception('Weird! All confidence levels are same!')
        target = select_range[np.random.randint(len(select_range))]
    print(label, label_predicted, target)
    dict_param['target'] = target
    dict_param['label_original'] = label
    dict_param['label_predicted'] = label_predicted
    dict_param['x'] = x

# Build MIP model

In [4]:
class MinistExtendedFormulation(object):
    def __init__(self, dict_param, is_conv=False, is_integral=False,
                 is_branchPriority=False, obj_type=GRB.MINIMIZE, pixel_change_tol=784, model_attrs={}):
        self.model = gp.Model("robustness_BNN")
        self.param = dict_param
        self.is_conv = is_conv
        self.is_integral= is_integral
        self.is_branchPriority = is_branchPriority
        self.obj_type = obj_type
        self.pixel_change_tol = pixel_change_tol
        self.model_attrs = model_attrs
        
        self.vars = {}
        self.vars_incumbent = dict(Status=False)
        self.confidence_level = 0.
        self.result_only = True # save the result only
    
    def setup(self):
        """set up the corresponding model
        """
        self.model.setParam("TimeLimit", 1800)        
        self.model.setParam("PreCrush", 1)
        #if self.is_integral is False:
            #choose barrier method for continuous models
            #self.model.setParam("Method", 2)
            #Disable crossover to accelarate the algorithm
            #self.model.setParam("Crossover", 0)
            
        for key,value in self.model_attrs.items():
            self.model.setParam(key, value)
            
        model = self.model
        w0,w1,w2= (self.param[name] for name in ['w0','w1','w2'])
        x = self.param['x']
        
        model._param = self.param
        model._num_usercut = 0
        
        size_input = self.param['size_input']
        size_hidden = self.param['size_hidden']
        size_output = self.param['size_output']
        
        label_predicted = dict_param['label_predicted']
        target = self.param['target']
        
        type_var = GRB.BINARY if self.is_integral else GRB.CONTINUOUS
        
        x0 = model.addVars(range(size_input), vtype=type_var, ub=1., name='x0')        
        x1 = model.addVars(range(size_hidden[0]), vtype=type_var, ub=1., name='x1')
        x2 = model.addVars(range(size_hidden[1]), vtype=type_var, ub=1., name='x2')     
        if self.is_branchPriority is True:
            # branch on lower-lever variables first
            priority = 1
            for i in x2:
                x2[i].branchPriority = priority
            priority = priority + 1
            for i in x1:
                x1[i].branchPriority = priority
            priority = priority + 1
            for i in x0:
                x0[i].branchPriority = priority
        
        y0 = model.addVars(range(size_input), lb=-1., ub=1., vtype=GRB.CONTINUOUS, name='y0')
        y1 = model.addVars(range(size_hidden[0]), lb=-1., ub=1., vtype=GRB.CONTINUOUS, name='y1')
        y2 = model.addVars(range(size_hidden[1]), lb=-1., ub=1., vtype=GRB.CONTINUOUS, name='y2')
        
        self.vars['y0'] = y0
        self.vars['y1'] = y1
        self.vars['y2'] = y2
        model._vars = self.vars
        #self.vars['z0'] = z0
        
        model.addConstrs(y0[j] <= 2*x0[j] - 1. for j in range(size_input))
        model.addConstrs(y1[j] <= 2*x1[j] - 1. for j in range(size_hidden[0]))
        model.addConstrs(y2[j] <= 2*x2[j] - 1. for j in range(size_hidden[1]))
        
        model.addConstrs(y0[j] >= 2*x0[j] - 1. for j in range(size_input))
        model.addConstrs(y1[j] >= 2*x1[j] - 1. for j in range(size_hidden[0]))
        model.addConstrs(y2[j] >= 2*x2[j] - 1. for j in range(size_hidden[1]))
        
        if self.is_conv is True:
            self.add_constrs_conv()
        else:
            self.add_constrs_natural()
            
        if self.obj_type == GRB.MINIMIZE:
            model.addConstrs(gp.quicksum(w2[j, k]*y2[k] for k in range(size_hidden[1])) <= 
                          gp.quicksum(w2[target, k]*y2[k] for k in range(size_hidden[1])) for j in range(size_output) if j != target)
            model.setObjective(size_input/2-gp.quicksum(x[j]*y0[j]/2 for j in range(size_input)), GRB.MINIMIZE)
        elif self.obj_type == GRB.MAXIMIZE:
            model.addConstr(size_input-gp.quicksum(x[j]*y0[j] for j in range(size_input)) <= 2*self.pixel_change_tol)
            model.setObjective(gp.quicksum(w2[target, k]*y2[k] for k in range(size_hidden[1])) - 
                               gp.quicksum(w2[label_predicted, k]*y2[k] for k in range(size_hidden[1])), GRB.MAXIMIZE)
    
    def set_confidence_level(self, confidence_level):
        self.confidence_level = confidence_level
        self.model = gp.Model("robustness_BNN")
        self.vars_incumbent = dict(Status=False)
        
    def add_constrs_natural(self):
        model = self.model
        size_hidden = self.param['size_hidden'] 
        size_input = self.param['size_input'] 
        w0,w1 = (self.param[name] for name in ['w0','w1'])
        y0,y1,y2= (self.vars[name] for name in ['y0','y1','y2'])
        # for each hidden layer, denote t as the output vector, y as the input vector and (row, col) as the shape of the weight matrix
#         for t,y,weights,row,col in zip((y1, y2), (y0, y1), (w0, w1), size_hidden, (size_input, size_hidden[0])):
#             #print(w)
#             #pdb.set_trace()
#             name = name='h'+str(col)+'L'
#             model.addConstrs((gp.quicksum(weights[i,j]*y[j] for j in range(col)) + col >= col * t[i] for i in range(row)), name=name)
#             name='h'+str(col)+'U'
#             model.addConstrs((gp.quicksum(weights[i,j]*y[j] for j in range(col)) - col + 1. <= col * y1[i] for i in range(row)), name=name)
            
        model.addConstrs(gp.quicksum(w0[i,j]*y0[j] for j in range(size_input)) + size_input
                         >= size_input * y1[i] for i in range(size_hidden[0]))
        model.addConstrs(gp.quicksum(w0[i,j]*y0[j] for j in range(size_input)) - size_input + 1.
                         <= size_input * y1[i] for i in range(size_hidden[0]))
        
        model.addConstrs(gp.quicksum(w1[i,j]*y1[j] for j in range(size_hidden[0])) + size_hidden[0]
                         >= size_hidden[0] * y2[i] for i in range(size_hidden[1]))
        model.addConstrs(gp.quicksum(w1[i,j]*y1[j] for j in range(size_hidden[0])) - size_hidden[0] + 1.
                         <= size_hidden[1] * y2[i] for i in range(size_hidden[1]))
        
    def add_constrs_conv(self):
        model = self.model
        size_hidden = self.param['size_hidden'] 
        size_input = self.param['size_input'] 
        w0,w1 = (self.param[name] for name in ['w0','w1'])
        y0,y1,y2= (self.vars[name] for name in ['y0','y1','y2'])
        # for each hidden layer, denote t as the output vector, y as the input vector and (row, col) as the shape of the weight matrix
        for t,y,w,row,col in zip((y1, y2), (y0, y1), (w0, w1), size_hidden, (size_input, size_hidden[0])):
            z_aux = model.addVars(range(row), range(col), lb=-1.0, vtype=GRB.CONTINUOUS)
            model.addConstrs(-(1.+t[i])<=2*z_aux[i,j] for i in range(row) for j in range(col))
            model.addConstrs(2*z_aux[i,j]<=(1.+t[i]) for i in range(row) for j in range(col))
            model.addConstrs(-(1-t[i])<=2*(w[i,j]*y[j]-z_aux[i,j]) for i in range(row) for j in range(col))
            model.addConstrs(2*(w[i,j]*y[j]-z_aux[i,j])<=(1-t[i]) for i in range(row) for j in range(col))
            model.addConstrs(z_aux.sum(i, '*')>=0 for i in range(row))
            model.addConstrs(gp.quicksum(w[i,j]*y[j] for j in range(col))-z_aux.sum(i,'*')<=(t[i]-1) for i in range(row))
    def __repr__(self):
        status = self.vars_incumbent['Status']
        s = ''
        if status is False:
            s = s + 'The optimization problem has not been solved. Please try to solve the problem first.'
        elif status != GRB.OPTIMAL:
            s = s + 'The solver is not able to solve the problem to optimality.'
        else:
            s = s + ('The solver is able to solve the problem to optimality within {}.\n'.format(self.vars_incumbent['time']))
            s = s + ('The original input is:\n{}\n'.format(self.param['x']))
            y0,y1,y2 = (self.vars_incumbent[name] for name in ['y0', 'y1', 'y2'])
            s= s + ('The targeted input is:\n{}\n'.format(y0))
            s= s + ('The hidden layers are:\n{},\n\n{},\n\n{}\n'.format(y1, y2))
        return s

    def __call__(self, callback=None):
        if self.vars_incumbent['Status'] is False:
            self.setup()          
            
        if callback is None:
            self.model.optimize()
        elif self.is_integral is False:
            self.model.setParam("OutputFlag", 0)
            self.model._tol = self.model.Params.FeasibilityTol
            #self.model._tol = 1e-5
            self.model._iter_tol = 10000
            self.model._iter = 0
            #self.model._lazy_status = False
            obj_previous = -1e5
            obj_current = 1e5
            while np.abs(obj_previous-obj_current)>self.model._tol*self.model._tol:
            #while (self.model._lazy_status is False) and (np.abs(obj_previous-obj_current)<=self.model._tol):
                self.model.optimize()
                obj_previous = obj_current
                obj_current = self.model.objVal
                callback(self.model)
        else:
            self.model._tol = 0
            self.model.optimize(callback)
            #self.model.optimize(conv_cb)
        self.vars_incumbent['Status'] = self.model.getAttr('Status')
        if self.vars_incumbent['Status'] == GRB.OPTIMAL:
            print('The objective is {}'.format(self.model.objVal))
            self.set_optimal_sols()
        
    def set_optimal_sols(self, is_print=True):
        model = self.model
        size_input = self.param['size_input']
        size_hidden = self.param['size_hidden'] 
        names_vars = ['y0','y1','y2']
        y0,y1,y2= (self.vars[name] for name in names_vars)
        self.vars_incumbent['y0'] = np.array(model.getAttr("X", y0).values())
        self.vars_incumbent['y1'] = np.array(model.getAttr("X", y1).values())
        self.vars_incumbent['y2'] = np.array(model.getAttr("X", y2).values())
        self.vars_incumbent['objVal'] = model.objVal
        self.vars_incumbent['time'] = model.getAttr('Runtime')
    
    def save(self, filename, result_only=True):
#         if self.vars_incumbent['Status'] == GRB.OPTIMAL:
#             model = self.model
#             x = self.param['x']
#             target = self.param['target']
#             names_incumbent = ['y0','y1','y2', 'objVal','time', 'Status']
#             y0,y1,y2,y3,objVal,time,status = (self.vars_incumbent[name] for name in names_incumbent)
#             dict_instance = {'x':x, 'y0':y0, 'y1':y1, 'y2':y2, 
#                          'target':target, 'time':time, 'Status':status,  'objVal':objVal}
#             # model
#             model.write(filename+'.mps')
#             # MIP start
#             model.write(filename+'.mst')
#             # save solutinos
#             model.write(filename+'.json')
#             # save parameters
#             with open(filename, 'wb') as f:
#                 pickle.dump(dict_instance, f)
            
#         else:
#             raise Exception('The status is not OPTIMAL!')
        
        x = self.param['x']
        target = self.param['target']
        label_original = self.param['label_original']
        label_predicted = self.param['label_predicted']
        names_incumbent = ['y0','y1','y2', 'objVal','time', 'Status']
        dict_instance = {'confidence_level':self.confidence_level, 'label_original':label_original,
                     'label_predicted':label_predicted, 'target':target, 'time':self.model.getAttr('Runtime')}
        
        try:
            names_incumbent = ['y0','y1','y2', 'objVal','time', 'Status']
            y0,y1,y2,objVal,time,status = (self.vars_incumbent[name] for name in names_incumbent)
            dict_instance.update({'x':x, 'y0':y0, 'y1':y1, 'y2':y2, 
                         'time':time, 'Status':status,  'objVal':objVal})
            model = self.model
        except:
            dict_instance['Status'] = -1
            
        if self.is_integral is True:
            try:
                dict_instance['gap'] = self.model.getAttr('MIPGap')
                dict_instance['bound'] = self.model.getAttr('ObjBound')
                dict_instance['objVal'] = self.model.getAttr('objVal')
                dict_instance['time'] = self.model.getAttr('Runtime')
                dict_instance['Status'] = self.model.getAttr('Status')
            except:
                print('MIP failed to save!')
                dict_instance = {'Status':-1}

        self.result_only = result_only
        if result_only is False:
            # model
            model.write(filename+'.mps')
            # MIP start
            model.write(filename+'.mst')
            # save solutinos
            model.write(filename+'.json')
            
            # save parameters
        with open(filename, 'wb') as f:
                pickle.dump(dict_instance, f)
            
    def load(self, filename):
        self.model = gp.read(filename+'.mps')
        size_input = self.param['size_input']
        size_hidden = self.param['size_hidden'] 
        with open(filename, 'rb') as f:
            dict_instance = pickle.load(f)
        self.param['x'] = dict_instance['x']
        self.param['target'] = dict_instance['target']
        names_incumbent = ['y0','y1','y2','objVal','time', 'Status']
        for name in names_incumbent:
            self.vars_incumbent[name] = dict_instance[name]
        self.model.read(filename+'.mst')
        self.vars['y0'] = dict(zip(range(size_input), ((x for x in instance2.model.getVars() if x.VarName.find('y0') != -1))))
        self.vars['y1'] = dict(zip(range(size_hidden[0]), ((x for x in instance2.model.getVars() if x.VarName.find('y1') != -1))))
        self.vars['y2'] = dict(zip(range(size_hidden[1]), ((x for x in instance2.model.getVars() if x.VarName.find('y2') != -1))))
        self.model.update()

In [5]:
def conv_cb(model, where):
    if where == GRB.Callback.MIPNODE:
        status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
        #if (status == GRB.OPTIMAL):
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)<= 100):
        # only adding cutting planes at the root node
        if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)==0):
            #pdb.set_trace()        
            size_hidden = model._param['size_hidden'] # number of constraints            
            size_input = model._param['size_input'] # number of variables
            w0,w1 = (model._param[name] for name in ['w0','w1'])
            y0,y1,y2= (model._vars[name] for name in ['y0','y1','y2'])
            for weights,y,y_out,m,n in zip((w0,w1), (y0,y1), (y1,y2), size_hidden, (size_input, size_hidden[0])):
                #pdb.set_trace()
                y_incumbent = np.array(model.cbGetNodeRel(y).select())
                y_out_incumbent = np.array(model.cbGetNodeRel(y_out).select())
                for i in range(m):
                    w = weights[i,:]
                    t = y_out_incumbent[i]
                    if np.sum(np.minimum(w*y_incumbent, t)) < n/2*(t-1) - model._tol:
                        indices = np.where(w*y_incumbent<t)[0]
                        # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                        model.cbCut((len(indices)-n/2)*y_out[i] - n/2 <= gp.quicksum(weights[i,j]*y[j] for j in indices))
                        model._num_usercut = model._num_usercut + 1

                    if np.sum(np.maximum(w*y_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
                        indices = np.where(w*y_incumbent>t)[0]
                        # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                        #pdb.set_trace()
                        model.cbCut((len(indices)-n/2+1)*y_out[i]+n/2-1 >= gp.quicksum(weights[i,j]*y[j] for j in indices))
                        model._num_usercut = model._num_usercut + 1

In [6]:
def add_cut(model):  
    size_hidden = model._param['size_hidden'] # number of constraints            
    size_input = model._param['size_input'] # number of variables
    w0,w1 = (model._param[name] for name in ['w0','w1'])
    y0,y1,y2= (model._vars[name] for name in ['y0','y1','y2'])
    model._iter = model._iter + 1
    for weights,y,y_out,m,n in zip((w0,w1), (y0,y1), (y1,y2), size_hidden, (size_input, size_hidden[0])):
        #pdb.set_trace()
        y_incumbent = np.array(model.getAttr("X", y).values())
        y_out_incumbent = np.array(model.getAttr("X", y_out).values())
        for i in range(m):
            w = weights[i,:]
            t = y_out_incumbent[i]
            if np.sum(np.minimum(w*y_incumbent, t)) < n/2*(t-1) - model._tol:
                indices = np.where(w*y_incumbent<t)[0]
                # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                model.addConstr((len(indices)-n/2)*y_out[i] - n/2 <= gp.quicksum(weights[i,j]*y[j] for j in indices))
                model._num_usercut = model._num_usercut + 1
                #model._lazy_status = False

            if np.sum(np.maximum(w*y_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
                indices = np.where(w*y_incumbent>t)[0]
                # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                #pdb.set_trace()
                model.addConstr((len(indices)-n/2+1)*y_out[i]+n/2-1 >= gp.quicksum(weights[i,j]*y[j] for j in indices))
                model._num_usercut = model._num_usercut + 1

# Experiments with varying changing tolerance


In [9]:
obj_conv = {}
obj_natural = {}
obj_conv_int = {}
obj_natural_int = {}

root_dir = 'experiment_results/robustness_integral_size_hidden2_max_with_varying_pixel_tol/'
for size_hidden in [256,]:
    data_dir = '2_layer_weights/' + str(size_hidden) + '/'
    dict_param, test_dataset = load_data(data_dir=data_dir)
    for idx_instance in [1,3,4]:
        set_param(dict_param, test_dataset)
        pixel_change_range = [0,1,2,3,4,5] # allowed to change about 5% of pixels
        print('Hidden size:{}, instance index:{}'.format(size_hidden, idx_instance+1)+'*'*40+'{}'.format(datetime.datetime.now()))
        for pixel_change_tol in pixel_change_range:
            print('Hidden size:{}, instance index:{}, tolerance:{}'.format(size_hidden, idx_instance+1, pixel_change_range)
                  +'*'*40+'{}'.format(datetime.datetime.now()))
            instance_natural = MinistExtendedFormulation(dict_param, is_integral=False,
                                                         obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL RELAXATION STARTS TO SOLVE'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural()
            try:
                instance_natural.save(root_dir+'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural[str(size_hidden), str(idx_instance)] = -1           


            instance_conv = MinistExtendedFormulation(dict_param, is_integral=False,
                                                      obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG RELAXATION STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv(add_cut)
            try:
                instance_conv.save(root_dir+'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))
                obj_conv[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv[str(size_hidden), str(idx_instance)] = -1

            instance_natural_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                             obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural_int()
            try:
                instance_natural_int.save(root_dir+'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural_int[str(size_hidden), str(idx_instance)] = -1

            instance_conv_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                          obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv_int(conv_cb)
            try:
                instance_conv_int.save(root_dir+'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_conv_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv_int[str(size_hidden), str(idx_instance)] = -1

8 8 7
Hidden size:256, instance index:2****************************************2021-05-16 01:13:38.290116
Hidden size:256, instance index:2, tolerance:[0, 1, 2, 3, 4, 5]****************************************2021-05-16 01:13:38.291114

 NATURAL RELAXATION STARTS TO SOLVE--------------------# hidden neurons256,#instance2,tol0

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 3617 rows, 2592 columns and 539472 nonzeros
Model fingerprint: 0x858cc067
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3361 rows and 2080 columns
Presolve time: 0.10s
Presolved: 256 rows, 768


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -4.17769    0   38  -22.00000   -4.17769  81.0%     -    0s
     0     0   -9.95238    0   43  -22.00000   -9.95238  54.8%     -    0s
H    0     0                     -18.0000000   -9.95238  44.7%     -    1s
     0     0  -18.00000    0   51  -18.00000  -18.00000  0.00%     -    1s

Cutting planes:
  User: 10
  Gomory: 4
  Cover: 1
  MIR: 6
  StrongCG: 1
  Zero half: 4
  RLT: 5

Explored 1 nodes (1094 simplex iterations) in 1.17 seconds
Thread count was 16 (of 16 available processors)

Solution count 2: -18 -22 
No other solutions better than -18

Optimal solution found (tolerance 1.00e-04)
Best objective -1.800000000000e+01, best bound -1.800000000000e+01, gap 0.0000%

User-callback calls 225, time in user-callback 0.39 sec
The objective is -18.0
Hidden size:256, instance index:2, tolerance:[0, 1, 2, 3, 4, 5]**********

     0     0   40.91648    0  119  -22.00000   40.91648   286%     -   14s
     0     0   39.94208    0   94  -22.00000   39.94208   282%     -   14s
     0     0   39.15088    0   94  -22.00000   39.15088   278%     -   15s
     0     0   38.98941    0   95  -22.00000   38.98941   277%     -   15s
     0     0   38.85341    0   94  -22.00000   38.85341   277%     -   15s
     0     0   38.78964    0   96  -22.00000   38.78964   276%     -   15s
     0     0   38.76588    0  105  -22.00000   38.76588   276%     -   15s
     0     0   38.75656    0  108  -22.00000   38.75656   276%     -   15s
     0     0   38.75299    0  106  -22.00000   38.75299   276%     -   15s
     0     0   37.11914    0   95  -22.00000   37.11914   269%     -   15s
     0     0   36.61568    0   97  -22.00000   36.61568   266%     -   15s
     0     0   36.45916    0  104  -22.00000   36.45916   266%     -   16s
     0     0   36.41385    0  107  -22.00000   36.41385   266%     -   16s
     0     0   36.39153  

  4009  1073    4.87399   22   80  -22.00000   11.82499   154%   102   60s
  5979  1719  -14.00000   24   43  -22.00000    7.77537   135%   105   65s
  7736  2296   -2.57492   27   70  -22.00000    6.56041   130%   107   70s
  9372  2838  -10.00000   29   58  -22.00000    4.51607   121%   108   75s
 11358  3392   -6.00000   30   53  -22.00000    3.53878   116%   110   80s
 13161  3805 infeasible   38       -22.00000    1.68649   108%   110   85s
 14557  3995  -14.02111   38   57  -22.00000    1.18584   105%   111   93s
 15093  4184   -2.84925   24   56  -22.00000    0.97369   104%   111   96s
 16522  4568     cutoff   26       -22.00000    0.48628   102%   111  101s
 18516  4849  -18.00000   33   35  -22.00000   -0.14345  99.3%   112  106s
 20178  5332  -10.23728   37   55  -22.00000   -1.58360  92.8%   111  110s
*20247  4917              39     -18.0000000   -1.58360  91.2%   111  110s
 21300  5198   -6.00000   29   56  -18.00000   -2.00000  88.9%   111  115s
 22913  5468  -10.00000  

     0     0   51.50295    0  175  -22.00000   51.50295   334%     -   90s
     0     0   51.47974    0  172  -22.00000   51.47974   334%     -   92s
     0     0   51.42783    0  161  -22.00000   51.42783   334%     -   93s
     0     0   51.39848    0  172  -22.00000   51.39848   334%     -   94s
     0     0   51.37635    0  170  -22.00000   51.37635   334%     -   95s
     0     0   51.35736    0  174  -22.00000   51.35736   333%     -   96s
     0     0   51.33940    0  170  -22.00000   51.33940   333%     -   98s
     0     0   51.31405    0  173  -22.00000   51.31405   333%     -   98s
     0     0   51.29641    0  179  -22.00000   51.29641   333%     -   99s
     0     0   51.27907    0  178  -22.00000   51.27907   333%     -  100s
     0     0   51.25107    0  178  -22.00000   51.25107   333%     -  101s
     0     0   51.21928    0  166  -22.00000   51.21928   333%     -  102s
     0     0   51.18610    0  175  -22.00000   51.18610   333%     -  103s
     0     0   51.13709  

...

     0     0   58.21254    0  137  -28.00000   58.21254   308%     -  498s
     0     0   58.20360    0  145  -28.00000   58.20360   308%     -  499s
     0     0   58.19821    0  137  -28.00000   58.19821   308%     -  500s
     0     0   58.19593    0  142  -28.00000   58.19593   308%     -  500s
     0     0   58.15700    0  141  -28.00000   58.15700   308%     -  502s
     0     0   58.12833    0  130  -28.00000   58.12833   308%     -  505s
     0     0   58.10939    0  138  -28.00000   58.10939   308%     -  506s
     0     0   58.10330    0  134  -28.00000   58.10330   308%     -  507s
     0     0   58.09917    0  139  -28.00000   58.09917   307%     -  508s
     0     0   58.07681    0  143  -28.00000   58.07681   307%     -  509s
     0     0   58.05963    0  132  -28.00000   58.05963   307%     -  510s
     0     0   58.04689    0  136  -28.00000   58.04689   307%     -  511s
     0     0   58.03312    0  138  -28.00000   58.03312   307%     -  512s
     0     0   58.02545  

     0     0   56.84658    0  123  -28.00000   56.84658   303%     -  616s
     0     0   56.83754    0  127  -28.00000   56.83754   303%     -  617s
     0     0   56.80382    0  129  -28.00000   56.80382   303%     -  618s
     0     0   56.79508    0  131  -28.00000   56.79508   303%     -  618s
     0     0   56.78741    0  132  -28.00000   56.78741   303%     -  619s
     0     0   56.78365    0  133  -28.00000   56.78365   303%     -  620s
     0     0   56.77359    0  137  -28.00000   56.77359   303%     -  621s
     0     0   56.75966    0  132  -28.00000   56.75966   303%     -  622s
     0     0   56.74846    0  146  -28.00000   56.74846   303%     -  622s
     0     0   56.73751    0  144  -28.00000   56.73751   303%     -  623s
     0     0   56.73280    0  142  -28.00000   56.73280   303%     -  624s
     0     0   56.66524    0  126  -28.00000   56.66524   302%     -  624s
     0     0   56.59349    0  111  -28.00000   56.59349   302%     -  625s
     0     0   56.57503  

     0     0   55.23665    0  118  -28.00000   55.23665   297%     -  734s
     0     0   55.22107    0  120  -28.00000   55.22107   297%     -  735s
     0     0   55.20189    0  118  -28.00000   55.20189   297%     -  736s
     0     0   55.19527    0  104  -28.00000   55.19527   297%     -  737s
     0     0   55.18418    0  111  -28.00000   55.18418   297%     -  738s
     0     0   55.17266    0  112  -28.00000   55.17266   297%     -  739s
     0     0   55.16577    0  114  -28.00000   55.16577   297%     -  740s
     0     0   55.15931    0  111  -28.00000   55.15931   297%     -  741s
     0     0   55.15829    0  122  -28.00000   55.15829   297%     -  742s
     0     0   55.15203    0  121  -28.00000   55.15203   297%     -  743s
     0     0   55.14904    0  117  -28.00000   55.14904   297%     -  745s
     0     0   55.14809    0  115  -28.00000   55.14809   297%     -  746s
     0     0   55.14672    0  112  -28.00000   55.14672   297%     -  747s
     0     0   55.14525  

    29    20 infeasible    6       -28.00000   41.06043   247%   254  890s
   385   218   28.56336   11   92  -28.00000   41.06043   247%   138  895s
   969   595   31.13403   13   79  -28.00000   41.06043   247%   140  900s
  1502   855    7.06212   21  112  -28.00000   41.06043   247%   133  916s
  1519   871   41.06043   16  110  -28.00000   41.06043   247%   133  924s
  1531   879   41.06043   18   90  -28.00000   41.06043   247%   135  925s
  2117  1005   11.60276   39   54  -28.00000   41.06043   247%   127  930s
  2948  1252   10.46561   34   66  -28.00000   41.06043   247%   126  935s
  3838  1489   25.30465   27   85  -28.00000   40.80412   246%   128  940s
  3914  1554   11.51796   34   64  -28.00000   40.80412   246%   127  959s
  4050  1514   -8.02031   53   48  -28.00000   40.80412   246%   126  967s
  4434  1801 infeasible   34       -28.00000   40.40575   244%   126  970s
  5797  2621 infeasible   37       -28.00000   39.38776   241%   122  975s
  7145  3574   14.08793  

 47686 18679   23.56219   48   91  -12.00000   30.91328   358%   148 1622s
 48377 18925    3.90153   52   91  -12.00000   30.73192   356%   149 1634s
 49176 19193   19.46710   47   90  -12.00000   30.61311   355%   150 1646s
 49967 19499    6.84051   54   98  -12.00000   30.50760   354%   150 1658s
 50873 19636   -8.00000   77   47  -12.00000   30.40417   353%   150 1672s
 51659 19794   19.60487   50   83  -12.00000   30.29582   352%   151 1686s
 52483 20026   11.76206   56   60  -12.00000   30.23427   352%   151 1700s
 53346 20220   22.83710   47   98  -12.00000   30.18001   352%   152 1715s
 54149 20429   25.33459   48  101  -12.00000   30.10075   351%   152 1730s
 54968 20672    7.78394   57   65  -12.00000   30.05927   350%   153 1742s
 55839 20394   15.90683   54   89  -12.00000   30.01916   350%   153 1769s
 55853 20927   15.78532   55   90  -12.00000   30.01549   350%   153 1782s
 56759 21108   13.86967   53   78  -12.00000   29.96942   350%   154 1795s
 57628 21000   14.64277  

In [14]:
def format_decimal(x):    
    if x==np.inf:
        s = '-'
    else:
        s = '{:.2f}'.format(x)
    return s
def format_int(x):
    if x==np.inf:
        s = '-'
    else:
        s = '{:d}'.format(int(x))
    return s
experiment_frame = pd.DataFrame({'dim_hidden':[], 'pixel_change_tol':[], 'label':[],
                                 'nat_relax':[], 'nat_obj':[], 'nat_bound':[], 'nat_gap':[], 'nat_time':[],
                                 'conv_relax':[], 'conv_obj':[], 'conv_bound':[], 'conv_gap':[], 'conv_time':[], 'rimp':[]})
root_dir = 'experiment_results/robustness_integral_size_hidden2_max_with_varying_pixel_tol/'
n=0
for size_hidden in [32, 64, 128, 256,]:
    dim_hidden = size_hidden
    pixel_change_range = [1,2,3,4,5]
    for pixel_change_tol in pixel_change_range:
        for idx_instance in range(5):
            filename = root_dir + 'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            label = (d['label_original'], d['label_predicted'], d['target'])
            nat_relax = d['objVal']

            filename = root_dir + 'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
                
            try:
                conv_relax = d['objVal']
            except:
                conv_relax = np.nan
#             if d['Status'] != -1:
#                 conv_relax = d['objVal']
#             else:
#                 conv_relax = np.nan

            filename = root_dir + 'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            nat_obj = d['objVal']
            nat_bound = d['bound']
            nat_gap = d['gap']*100
            nat_time = d['time']

            filename = root_dir + 'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            conv_obj = d['objVal']
            conv_bound = d['bound']
            conv_gap = d['gap']*100
            conv_time = d['time']
            rimp = 100*(nat_relax-conv_relax)/(nat_relax-nat_obj + (np.abs(nat_relax-nat_obj)<=1e-4))

            list_frame = ['','', label,  nat_relax,  nat_obj, nat_bound, nat_gap, nat_time,
                                       conv_relax, conv_obj, conv_bound, conv_gap, conv_time, rimp]
            if idx_instance == 0:
                list_frame[0] = dim_hidden
                list_frame[1] = pixel_change_tol

            experiment_frame.loc[n] = list_frame
            n= n + 1
        list_frame = ['', '', 'Ave'] + list(experiment_frame[-5:].mean())
        experiment_frame.loc[n] = list_frame
        n = n + 1
    
        
formatters = [None,  None, None,format_decimal, format_int, format_int, format_int, format_decimal, format_decimal,
              format_int, format_int, format_int, format_decimal, format_decimal]        
latex_str = experiment_frame.to_latex(index=False, formatters=formatters)
print(latex_str)

C:\Users\Shaoning\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


\begin{tabular}{lllrrrrrrrrrrr}
\toprule
dim\_hidden & pixel\_change\_tol &      label & nat\_relax & nat\_obj & nat\_bound & nat\_gap & nat\_time & conv\_relax & conv\_obj & conv\_bound & conv\_gap & conv\_time &  rimp \\
\midrule
        32 &                1 &  (0, 0, 7) &     32.00 &     -20 &       -20 &       0 &     0.04 &       2.46 &      -20 &        -20 &        0 &      0.05 & 56.82 \\
           &                  &  (0, 0, 2) &     30.00 &     -18 &       -18 &       0 &     0.04 &       7.52 &      -18 &        -18 &        0 &      0.05 & 46.84 \\
           &                  &  (8, 8, 0) &     24.00 &      -8 &        -8 &       0 &     0.04 &       6.34 &       -8 &         -8 &        0 &      0.04 & 55.19 \\
           &                  &  (3, 3, 5) &     10.00 &      -6 &        -6 &       0 &     0.04 &      -2.57 &       -6 &         -6 &        0 &      0.05 & 78.56 \\
           &                  &  (0, 0, 2) &     30.00 &     -10 &       -10 &       0 &    

In [16]:
experiment_frame.to_csv(root_dir+'results.csv', index=False)